In [55]:
%pip install transformers
%pip install tokenizers
%pip install bz2
%pip install torch
%pip install torch torchvision torchaudio
%pip install datasets




Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Minh Bao\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Minh Bao\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


^C
Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Minh Bao\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Users\Minh Bao\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Minh Bao\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
from transformers import pipeline
from transformers import AutoModel
from transformers import AutoTokenizer
import transformers
import torch
from bz2 import decompress
import requests
import zipfile
import bz2
import xml.etree.ElementTree as ET
import pandas as pd
from glob import glob
import json
import datasets


In [ ]:
def all_file_in_directory(path):
    res=[]
    for f in os.listdir(path):
        if os.path.isfile(path+"/"+f):
            res.append(f)
        elif os.path.isdir(f):
            base=path+"/"+f+""
            tmp=all_file_in_directory(base)
            for t in tmp:
                res.append(base+"/"+t)
    return res

    # return glob("*", recursive = True)

all_file_in_directory(".")

['./e2eqa-train+public_test-v1/zac2022_testa_only_question.json',
 './e2eqa-train+public_test-v1/zac2022_testa_sample_submission.json',
 './e2eqa-train+public_test-v1/zac2022_train_merged_final.json',
 'e2eqa-train+public_test-v1.zip',
 'Question-answering.ipynb',
 'viwiki-20220620-pages-articles.xml',
 'viwiki-20220620-pages-articles.xml.bz2',
 'wikipedia_20220620_all_titles.txt',
 'wikipedia_20220620_cleaned.jsonl',
 'wikipedia_20220620_cleaned.zip']

In [ ]:
train_merged_final='./e2eqa-train+public_test-v1/zac2022_train_merged_final.json'
testa_only_question='./e2eqa-train+public_test-v1/zac2022_testa_only_question.json'
testa_sample_submission='./e2eqa-train+public_test-v1/zac2022_testa_sample_submission.json'

In [30]:
def decompress_bz2_file(file_path, new_file_path):
  with open(new_file_path, 'wb') as new_file, bz2.BZ2File(file_path, 'rb') as file:
        for data in iter(lambda : file.read(100 * 1024), b''):
            new_file.write(data)

def decompress_zip_file(file_path):
  with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(".")

def download_and_save_file(url, compress_type, cache_file):
  file_name=url.split("/")[-1]
  if(os.path.exists(file_name)==True and cache_file==True):
    print("file ", file_name, "exists, cached")
  else:
    r = requests.get(url, allow_redirects=True)
    open(file_name, 'wb').write(r.content)
  if compress_type=="zip":
    decompress_zip_file(file_name)
  elif compress_type=="bz2":
    arr=file_name.split(".")
    arr.pop()
    new_file_name='.'.join(arr)
    decompress_bz2_file(file_name, new_file_name)

In [31]:
dump_url="https://dumps.wikimedia.org/viwiki/20220620/viwiki-20220620-pages-articles.xml.bz2"
cleaned_url="https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip"
data_url="https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip"

In [12]:
# download_and_save_file(dump_url, "bz2", True)
# download_and_save_file(cleaned_url, "zip", True)
# download_and_save_file(data_url, "zip", True)

file  viwiki-20220620-pages-articles.xml.bz2 exists, cached


In [33]:
# pho_model_name="vinai/phobert-base"
# pho_bert=transformers.AutoModel.from_pretrained(pho_model_name)
# pho_tokenizer = transformers.AutoTokenizer.from_pretrained(pho_model_name)

model_name="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
print('tokenizer.is_fast: ', tokenizer.is_fast)

tokenizer.is_fast:  True


In [7]:
# dump_tree=ET.parse("viwiki-20220620-pages-articles.xml")
# dump_root=dump_tree.getroot()
# print(dump_root)

In [45]:
def load_data_from_json(filename):
    f=open(filename, encoding="utf8")
    data=json.load(f)
    f.close()
    return data

data_merged_final=load_data_from_json(train_merged_final)["data"]
# print(data_merged_final)

In [50]:
def convert_list_to_datasetdict(list, columns):
    df = pd.DataFrame (list, columns=columns)
    dataset=datasets.Dataset.from_dict(df)
    return dataset

columns_merged_final = ['id', 'question', 'title', 'text', 'category', 'is_long_answer']
dataset_merged_final=convert_list_to_datasetdict(data_merged_final, columns_merged_final)
print(dataset_merged_final)

Dataset({
    features: ['id', 'question', 'title', 'text', 'category', 'is_long_answer'],
    num_rows: 20857
})


In [28]:
# list=data["data"]
# context=list[0]["text"]
# question=list[0]["question"]

# inputs = pho_tokenizer(
#     question,
#     context,
#     max_length=100,
#     truncation="only_second",
#     stride=50,
#     return_overflowing_tokens=True,
# )

# for ids in inputs["input_ids"]:
#     print(pho_tokenizer.decode(ids))

# print(inputs.keys())

< s >
T h ủ
t ư ớ n g
T r u n g
Q u ố c
l à
g ì
< / s >
< / s >
T h ủ
t ư ớ n g
T r u n g
Q u ố c
l à
n h â n
v ậ t
l ã n h
đ ạ o
c h í n h
p h ủ
,
c h ủ
t r ì
Q u ố c
v ụ
v i ệ n
g ồ m
b ố n
p h ó
t h ủ
t ư ớ n g
c ù n g
n g ư ờ i
đ ứ n g
đ ầ u
c á c
b ộ
v à
u ỷ
b a n
c ấ p
b ộ
.
C h ủ
t ị c h
n ư ớ c
đ ư ơ n g
n h i ệ m
l à
T ậ p
C ậ n
B ì n h
,
ô n g
c ũ n g
l à
T ổ n g
B í
t h ư
c ủ a
Đ ả n g
C ộ n g
s ả n
T r u n g
Q u ố c
v à
C h ủ
t ị c h
Q u â n
u ỷ
T r u n g
Q u ố c
,
d o
v ậ y
ô n g
l à
l ã n h
đ ạ o
t ố i
c a o
c ủ a
T r u n g
Q u ố c
.
< / s >
dict_keys(['overflowing_tokens', 'num_truncated_tokens', 'input_ids', 'token_type_ids', 'attention_mask'])


In [22]:
# inputs["overflowing_tokens"]

# pho_tokenizer.is_fast

False

In [51]:
question=dataset_merged_final[0]['question']
context=dataset_merged_final[0]['text']

inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

print(inputs.keys())

[CLS] Thủ tướng Trung Quốc là gì [SEP] Thủ tướng Trung Quốc là nhân vật lãnh đạo chính phủ, chủ trì Quốc vụ viện gồm bốn phó thủ tướng cùng người đứng đầu các [SEP]
[CLS] Thủ tướng Trung Quốc là gì [SEP] phủ, chủ trì Quốc vụ viện gồm bốn phó thủ tướng cùng người đứng đầu các bộ và [UNK] ban cấp bộ. Chủ tịch nước đương nhiệm l [SEP]
[CLS] Thủ tướng Trung Quốc là gì [SEP] tướng cùng người đứng đầu các bộ và [UNK] ban cấp bộ. Chủ tịch nước đương nhiệm là Tập Cận Bình, ông cũng là Tổng Bí thư của Đả [SEP]
[CLS] Thủ tướng Trung Quốc là gì [SEP]ộ. Chủ tịch nước đương nhiệm là Tập Cận Bình, ông cũng là Tổng Bí thư của Đảng Cộng sản Trung Quốc và Chủ tịch Quân [UNK] Trung Quốc [SEP]
[CLS] Thủ tướng Trung Quốc là gì [SEP], ông cũng là Tổng Bí thư của Đảng Cộng sản Trung Quốc và Chủ tịch Quân [UNK] Trung Quốc, do vậy ông là lãnh đạo tối cao của Trung Quốc. [SEP]
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])


In [52]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0, 0]

In [54]:
inputs = tokenizer(
    dataset_merged_final[2:6]['question'],
    dataset_merged_final[2:6]['text'],
    max_length=100,
    truncation="only_second",
    # stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 7 features.
Here is where each comes from: [0, 0, 1, 1, 2, 3, 3].
